In [1]:
from xml.dom import minidom
import re
import pandas as pd
import sys
import pickle
import numpy as np


from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()



sys.__stdout__=sys.stdout
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    #text=re.sub('\n',' ',text)
    #text=re.sub('-',' ',text)
    #text=re.sub('[\'"]','',text)
    #text=re.sub('[()?.@*#&!,$]','',text)
    text=re.sub('[^a-z0-9]',' ',text)
    return text#TAG_RE.sub('', text)


def make_dictionary(topics,max_doc,root):
    index=0
    wordmap={}
    word_count=[]
    for topic in topics:
        print(topic)
        mydoc = minidom.parse(root+topic+'.xml')
        docs = mydoc.getElementsByTagName('row')
        doc_count=0
        for doc in docs:
            if doc_count>=max_doc:
                break
            doc=doc.attributes['Body'].value
            doc=doc.lower()
            doc = remove_tags(doc)
            words=doc.split(' ')
            for word in words:
                word=ps.stem(word)
                if word in s_words:
                    continue
                if word not in wordmap:
                    wordmap[word]=index
                    word_count.append(int(1))
                    index=index+1
                elif word in wordmap:
                    ind=wordmap[word]
                    word_count[ind]=int(word_count[ind])+1
            doc_count=doc_count+1
    return wordmap,word_count


In [2]:
def vectorize_seq_nb(topics,max_doc,wordmap,root,doc_per_vect='all'):
    if doc_per_vect=='single':
        n_row=len(topics)*(max_doc)
    if doc_per_vect=='all':
        n_row=len(topics)
    vectors=np.zeros((n_row,len(wordmap)),dtype=np.float64)
    vct_label=[]
    row=0
    _V=len(wordmap)
    for topic in topics:
        print(topic)
        mydoc = minidom.parse(root+topic+'.xml')
        docs = mydoc.getElementsByTagName('row')
        doc_count=0
        for doc in docs:
            if doc_count>=max_doc:
                break
            #vector = [0]*(len(wordmap)+2)
            doc=doc.attributes['Body'].value
            doc=doc.lower()
            doc = remove_tags(doc)
            words=doc.split(' ')
                     
            if len(words)<5:
                continue
            for word in words:
                word=ps.stem(word)
                if word in wordmap:
                    ind=wordmap[word]
                    vectors[row,ind]=vectors[row,ind]+1
            doc_count=doc_count+1
            word_list.append(words)
            if doc_per_vect=='single':
                vct_label.append(topic)
                row=row+1
        if doc_per_vect=='all':
            vct_label.append(topic)
            #total=np.sum(vectors[row])
            #vectors[row]=(vectors[row]+alpha)/(total+alpha*_V)
            row=row+1
    return vectors,vct_label

In [3]:
root='Dataset/Training/'
root_test='Dataset/Test/'
f=open('Stopwords.txt')
s_words=f.read()
s_words=s_words.split('\n')
s_words.append('')

topics=['3d_Printer','Coffee','Arduino','Astronomy','Biology']#,'Chess','Cooking','Law','Space','Windows_Phone','Wood_Working']
#topics=['Anime']


 
#wordmap=make_dictionary(topics,100,root)
#vectors=vectorize_seq(topics,100,wordmap,root)
word_list=[]
write_backup=1
max_docs_train=200
if write_backup==1:
    wordmap,wc=make_dictionary(topics,max_docs_train,root)
    vectors,vct_labels=vectorize_seq_nb(topics,max_docs_train,wordmap,root)
    pickle_out = open("wordmap_test.pickle","wb")
    pickle.dump(wordmap, pickle_out)
    pickle_out.close()
    pickle_out = open("vectors_test.pickle","wb")
    pickle.dump(vectors, pickle_out)
    pickle_out.close()

read_backup=0
if read_backup==1:
    print('Reading backup')
    pickle_read = open("wordmap_500.pickle","rb")
    wordmap=pickle.load(pickle_read)
    pickle_read.close()

    pickle_read = open("vectors_500.pickle","rb")
    vectors=pickle.load(pickle_read)
    pickle_read.close()

#words=list(wordmap.keys())
#freq=list(wordmap.values())
#word_df=pd.DataFrame({'words':words,'freq':freq})
#word_df=word_df.sort_values('freq',ascending=False)
    



3d_Printer
Coffee
Arduino
Astronomy
Biology
3d_Printer
Coffee
Arduino
Astronomy
Biology


In [4]:
from matplotlib import pyplot as plt
print((vectors[0]))
x=range(len(vectors[0]))
plt.plot(x,vectors[0])
plt.plot(x,vectors[2])
#plt.plot(x,vectors[2])
plt.show()
print(vct_labels)
print(len(wordmap))
#no stem 500 20599 
#stem 500 14883

[ 35. 446.  38. ...   0.   0.   0.]


<Figure size 640x480 with 1 Axes>

['3d_Printer', 'Coffee', 'Arduino', 'Astronomy', 'Biology']
8384


In [9]:
max_docs_test=60
test_vectors,true_lbs=vectorize_seq_nb(topics,max_docs_test,wordmap,root_test,doc_per_vect='single')
def nb(test_vector,vectors,vct_labels,alpha):
    probs=[]
    maxm=0
    lb=''
    prob=0
    vct=0
    _vectors=vectors.copy()
    for row in range(len(_vectors)):
        len_of_wordmap=len(wordmap)
        total_word=np.sum(_vectors[row])
        _vectors[row]=(_vectors[row]+alpha)/(total_word+alpha*len_of_wordmap)
    for row in range(len(_vectors)):
        vect=_vectors[row]
        vect[test_vector==0]=1
        #test_vector_zero_mask=np.array(test_vector==0,dtype=np.uint8)
        #test_vector_one_mask=np.array(test_vector!=0,dtype=np.uint8)
        #vect=test_vector_one_mask*vectors[row]+test_vector_zero_mask
        
        prob=np.prod(vect)
        probs.append(prob)
        if prob>maxm:
            maxm=prob
            lb=vct_labels[row]
    return probs,lb
print(len(test_vectors[0]))

3d_Printer
Coffee
Arduino
Astronomy
Biology
8384


In [28]:
import time
alph=10
no_topics=len(topics)
tot_doc=no_topics*max_docs_train
tot_doc_test=no_topics*max_docs_test
acc_log=''
acc_log+='******************************************************************\n'
acc_log+=str(topics)+'\n'
acc_log+=str(['no_topic','train doc per topic','train_doc','test doc per topic','test_doc','word map'])+'\n'
acc_log+=str([no_topics,max_docs_train,tot_doc,max_docs_test,tot_doc_test,str(len(wordmap))])+'\n'
acc_log+='******************************************************************\n'
acc_log+=str(['alpha','accuracy','time'])+'\n'
for al in range(101):
    T=time.time()
    acc=0
    for i in range(len(test_vectors)):
        alpha=al/100
        pred,lb=nb(test_vectors[i],vectors,vct_labels,alpha)
        if lb==true_lbs[i]:
            acc=acc+1
        #print('res','pred',lb,'true',true_lbs[i],i+1,acc)
    T=time.time()-T
    accuracy=acc/i
    print('alpha: ',alpha,'acc :',accuracy)
    
    acc_log+=(str([alpha,accuracy,T])+'\n')
acc_log=re.sub('[\[\]]','',acc_log)
f=open('nb_log.csv','a')
f.write(acc_log)
f.close()
    #print(pred)

alpha:  0.0 acc : 0.1705685618729097
alpha:  0.01 acc : 0.8060200668896321
alpha:  0.02 acc : 0.8093645484949833
alpha:  0.03 acc : 0.8093645484949833
alpha:  0.04 acc : 0.8127090301003345
alpha:  0.05 acc : 0.8193979933110368
alpha:  0.06 acc : 0.822742474916388
alpha:  0.07 acc : 0.822742474916388
alpha:  0.08 acc : 0.822742474916388
alpha:  0.09 acc : 0.822742474916388
alpha:  0.1 acc : 0.822742474916388
alpha:  0.11 acc : 0.822742474916388
alpha:  0.12 acc : 0.8260869565217391
alpha:  0.13 acc : 0.8260869565217391
alpha:  0.14 acc : 0.8260869565217391
alpha:  0.15 acc : 0.822742474916388
alpha:  0.16 acc : 0.822742474916388
alpha:  0.17 acc : 0.822742474916388
alpha:  0.18 acc : 0.822742474916388
alpha:  0.19 acc : 0.822742474916388
alpha:  0.2 acc : 0.822742474916388
alpha:  0.21 acc : 0.822742474916388
alpha:  0.22 acc : 0.822742474916388
alpha:  0.23 acc : 0.822742474916388
alpha:  0.24 acc : 0.822742474916388
alpha:  0.25 acc : 0.822742474916388
alpha:  0.26 acc : 0.82274247491

In [67]:
246/300

0.82

In [64]:
#print(word_list)
d=np.array([1,2,3,4,5,6])
print(d)
k=d.copy()
k[k==3]=100
print(d,k)
tv=vectors
a=vectors[0,:]
#print(len())
print(np.array([k==2],dtype=np.uint8))
var=8
print(id(var))
def v(var):
    var=var+1
    print(id(var))
    return var
var=v(var)
print(id(var))
v=np.ones(5)
def func(v):
    v[2]=v[2]+1
    return 1
func(v)
print(v)

[1 2 3 4 5 6]
[1 2 3 4 5 6] [  1   2 100   4   5   6]
[[0 1 0 0 0 0]]
10910624
10910656
10910656
[1. 1. 2. 1. 1.]


In [38]:
words=list(wordmap.keys())
freq=list(wordmap.values())
word_df=pd.DataFrame(words,index=list(wordmap.values()))
word_df=word_df.sort_index()
word_df['freq']=wc
word_df=word_df.sort_values('freq',ascending=False)
print(word_df[0:50])
#print(len(vectors))

              0  freq
216         thi  2624
8           use  2152
4005      coffe  1773
1999       code  1694
1848    arduino  1190
0         print   929
90     question   800
43         time   784
258          ha   694
37          ani   642
77       differ   638
601   blockquot   628
404       water   522
49         onli   504
141        work   496
657          wa   487
379         way   483
2112        amp   482
5179       star   472
13      printer   461
2304        pin   458
66         veri   449
4232       brew   436
2269      board   433
2249        pre   425
123      filter   394
173         doe   388
84     filament   379
886        cell   367
954       stack   360
3575     serial   356
55       becaus   349
208     process   348
4228       bean   346
170        look   345
296       gener   345
4289    caffein   344
9649     planet   343
955       imgur   335
291        good   332
356      exampl   328
112        mani   324
578         bit   323
346         tag   318
174       

In [23]:
from matplotlib import pyplot as plt
vct=[vectors[12][:500]]
test=vectors[15][:498]
dis=eucl_dis(test,vct)
print(vct)
print(test)
print(dis[0][0])
x=range(0,500)
xt=range(0,498)
plt.plot(x,vct[0],xt,test)
plt.show

IndexError: index 12 is out of bounds for axis 0 with size 5

In [268]:
def eucl_dis(test_vector,vectors):
    h_dists=[]
    for vector in vectors:
        vect=np.array(vector[:-2])
        vect[vect>1]=1
        label=vector[-1]
        test_vect=np.array(test_vector)
        dist=np.abs(vect-test_vect)
        #dist=np.sum(dist)
        h_dists.append([dist,label])
    return h_dists

test_vector=vectors[0][:-2]
vecto=[vectors[0],vectors[1]]
h_dists=cos_dis(test_vector,vecto)
#h_df=pd.DataFrame(h_dists)
#h_df=h_df.sort_values(0)
print(h_dists)
#print((vectors[15]))

[[0.0, 'Coffee'], [4.1408162514693805, 'Coffee']]


In [109]:
raw='I am hasib\n,i "am" a student() so? @i do& my-duties! porp. and' 
#text=re.sub('\n',' ',raw)
text=raw.lower()
text=re.sub('[^a-z0-9]',' ',text)
#text=re.sub('-',' ',text)
#text=re.sub('[\'"]','',text)
#text=re.sub('[()?.@*#&!,$+]','',text)
#print(text.split(' '))

In [27]:
txt=(str([1,2,3]))
txt=re.sub('[\[\]]','',txt)
print(txt)

1, 2, 3
